In [60]:
import pandas as pd
import copy

In [66]:
character_stats = pd.read_parquet('dfs/character_stats.parquet')
character_stats

,characters,speed,acceleration,weight,handling,traction/grip,mini-turbo,total
0,"Baby Peach, Baby Daisy",2.25,4.00,2.00,5.00,4.25,4.00,21.50
1,"Baby Rosalina, Lemmy Koopa",2.25,4.25,2.00,4.75,3.75,4.00,21.00
2,"Baby Mario/Luigi, Dry Bones",2.50,4.25,2.00,4.50,4.00,3.75,21.00
3,"Koopa Troopa, Lakitu, Bowser Jr.",2.75,4.00,2.50,4.50,4.25,3.75,21.75
4,"Toadette, Wendy, Isabelle",2.75,4.25,2.50,4.25,3.50,3.75,21.00
5,"Toad, ShyGuy, Larry",3.00,4.00,2.75,4.25,4.00,3.50,21.50
6,"Cat Peach, Inkling Girl, Villager Girl",3.25,4.00,2.75,4.00,3.75,3.50,21.25
7,"Peach, Daisy, Yoshi",3.50,3.75,3.00,3.75,3.75,3.50,21.25
8,"Tanooki Mario, Inkling Boy, Villager Boy",3.50,3.75,3.25,3.75,3.25,3.50,21.00
9,"Luigi, Iggy",3.75,3.50,3.50,3.75,3.25,3.25,21.00


In [69]:
series = []
for characters in character_stats.characters:
    for character in characters.split(','):
        stats = character_stats[character_stats['characters']==characters].to_dict()
        data = {
            'name': character.strip(),
            'speed': float(list(stats['speed'].values())[0]),
            'acceleration': float(list(stats['acceleration'].values())[0]),
            'weight': float(list(stats['weight'].values())[0]),
            'handling': float(list(stats['handling'].values())[0]),
            'traction/grip': float(list(stats['traction/grip'].values())[0]),
            'mini-turbo': float(list(stats['mini-turbo'].values())[0])
        }
        # handle Baby Mario and Luigi
        if data['name'] == 'Baby Mario/Luigi':
            data2 = copy.deepcopy(data)
            data['name'] = 'Baby Luigi'
            data2['name'] = 'Baby Mario'
            series.append(pd.Series(data2))
        # handle Metal and Gold Mario
        if data['name'] == 'Metal/Gold Mario':
            data2 = copy.deepcopy(data)
            data['name'] = 'Gold Mario'
            data2['name'] = 'Metal Mario'
            series.append(pd.Series(data2))
        # add champion link
        if data['name'] == 'Link':
            data2 = copy.deepcopy(data)
            data2['name'] = 'Champion Link'
            series.append(pd.Series(data2))
        # handle yoshi colors
        if data['name'] == 'Yoshi':
            yoshi_colors = ['Black','Blue']
            for color in yoshi_colors:
                data2 = copy.deepcopy(data)
                data2['name'] = f'{color} Yoshi'
                series.append(pd.Series(data2))
        # handle Lemy
        if data['name'] == 'Lemmy Koopa':
            data['name'] = 'Lemmy'

        series.append(pd.Series(data))

# add things not in the table
# add miis with costumes
costumes = ['',' Crossing', ' DK', ' Pac-Man', ' Bowser']
for costume in costumes:
    data = {
        'name': f'Heavy Mii{costume}',
        'speed': 4.25,
        'acceleration': 2.25,
        'weight': 4.25,
        'handling': 2.75,
        'traction/grip': 3.5,
        'mini-turbo': 1.75
    }
    series.append(pd.Series(data))



pd.DataFrame(series).to_parquet('dfs/character_stats_adjusted.parquet')

